In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
%matplotlib inline

In [2]:
epl_matches_train = pd.read_csv('epl_matches_train.csv')
epl_matches_test = pd.read_csv('epl_matches_test.csv')
epl_players = pd.read_csv('epl_players.csv')
epl_teams = pd.read_csv('epl_teams.csv')
epl_goals = pd.read_csv('epl_goals.csv')
epl_potential_shots = pd.read_csv('epl_potential_shots.csv')

In [3]:
epl_matches_train

,season,stage,date,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,...,red_card_home_team,red_card_away_team,crosses_home_team,crosses_away_team,corner_home_team,corner_away_team,possession_home_team,possession_away_team,home_team_goal,away_team_goal
0,2008/2009,1,2008-08-17 00:00:00,49337,10260,10261,1,2,4,6,...,0,0,24,9,6,6,55.0,45.0,1,1
1,2008/2009,1,2008-08-16 00:00:00,38136,9825,8659,1,2,4,6,...,0,0,21,7,7,5,66.0,34.0,1,0
2,2008/2009,1,2008-08-16 00:00:00,43276,8472,8650,1,2,4,6,...,0,0,15,19,1,8,46.0,54.0,0,1
3,2008/2009,1,2008-08-16 00:00:00,40671,8654,8528,1,2,4,6,...,0,0,15,27,6,10,52.0,48.0,2,1
4,2008/2009,1,2008-08-17 00:00:00,34633,10252,8456,1,2,4,6,...,0,0,16,16,7,8,52.0,48.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2014/2015,9,2014-10-25 00:00:00,48955,8472,9825,1,2,4,6,...,0,0,14,21,3,3,39.0,61.0,0,2
2656,2014/2015,9,2014-10-25 00:00:00,38293,10003,8197,1,2,4,6,...,0,0,10,10,2,2,49.0,51.0,2,0
2657,2014/2015,9,2014-10-26 00:00:00,40230,8586,10261,1,2,4,6,...,0,0,29,11,10,2,68.0,32.0,1,2
2658,2014/2015,9,2014-10-25 00:00:00,45930,8659,9826,1,2,4,6,...,0,0,27,7,6,3,69.0,31.0,2,2


In [4]:
epl_matches_test

,season,stage,date,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,2015/2016,1,8/9/2015 0:00,48994,9825,8654,1,2,4,6,...,109897.0,35110,49543,155782,37169,575789,148302,25496,18506,192899
1,2015/2016,1,8/8/2015 0:00,54777,8678,10252,1,2,4,6,...,183500.0,24208,161414,473853,261313,179410,182223,23991,154280,23264
2,2015/2016,1,8/8/2015 0:00,31434,8455,10003,1,2,4,6,...,111800.0,155050,24948,102356,127130,144996,95955,157729,52563,26344
3,2015/2016,1,8/8/2015 0:00,56248,8668,9817,1,2,4,6,...,213809.0,41927,40548,35712,41365,30966,24915,37411,71724,72436
4,2015/2016,1,8/8/2015 0:00,44311,8197,8472,1,3,5,7,...,22964.0,26108,165526,180330,35443,25150,109058,24159,30348,42598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2015/2016,9,10/17/2015 0:00,42896,8466,8197,1,2,4,6,...,49571.0,23571,38899,43061,139671,173317,319300,214570,20694,286119
376,2015/2016,9,10/19/2015 0:00,49098,10003,10194,1,2,4,6,...,34036.0,119541,200962,37194,23253,39109,176300,96643,110148,172321
377,2015/2016,9,10/17/2015 0:00,55212,8586,8650,1,2,4,6,...,157838.0,22764,94043,314605,95327,307021,38807,184536,37234,426202
378,2015/2016,9,10/17/2015 0:00,54075,9817,9825,1,2,4,6,...,427438.0,35606,46539,38521,159594,37436,75489,36378,50047,31013


In [5]:
pd.set_option('display.max_columns',None)

In [6]:
def gen_result(df):
    l = []
    f= lambda x,y: 2 if x>y else (1 if x==y else 0)
    home_goals = list(df.home_team_goal)
    away_goals = list(df.away_team_goal)
    m = len(home_goals)
    for i in range(m):
        l.append(f(home_goals[i],away_goals[i]))
    df['results'] = l
    return df

In [7]:
epl_matches_train = gen_result(epl_matches_train)
epl_matches_train

,season,stage,date,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,on_target_shot_home_team,on_target_shot_away_team,off_target_shot_home_team,off_target_shot_away_team,foul_home_team,foul_away_team,yellow_card_home_team,yellow_card_away_team,red_card_home_team,red_card_away_team,crosses_home_team,crosses_away_team,corner_home_team,corner_away_team,possession_home_team,possession_away_team,home_team_goal,away_team_goal,results
0,2008/2009,1,2008-08-17 00:00:00,49337,10260,10261,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,2,4,6,8,5,5,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,9,11,30726,30362.0,30620,30865,32569.0,24148,34944.0,30373.0,24154.0,24157.0,30829.0,24224,25518.0,24228.0,30929,29581.0,38807.0,40565.0,30360.0,33852.0,34574.0,37799.0,11,1,10,9,16,11,3,0,0,0,24,9,6,6,55.0,45.0,1,1,1
1,2008/2009,1,2008-08-16 00:00:00,38136,9825,8659,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,5,7,9,1,3,5,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,7,11,23686,26111.0,38835,30986,31291.0,31013,30935.0,39297.0,26181.0,30960.0,36410.0,36373,36832.0,23115.0,37280,24728.0,24664.0,31088.0,23257.0,24171.0,25922.0,27267.0,12,2,13,3,11,9,0,0,0,0,21,7,7,5,66.0,34.0,1,0,2
2,2008/2009,1,2008-08-16 00:00:00,43276,8472,8650,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,10,10,32562,38836.0,24446,24408,36786.0,38802,24655.0,17866.0,30352.0,23927.0,24410.0,30660,37442.0,30617.0,24134,414792.0,37139.0,30618.0,40701.0,24800.0,24635.0,30853.0,4,11,3,5,13,12,0,2,0,0,15,19,1,8,46.0,54.0,0,1,0
3,2008/2009,1,2008-08-16 00:00:00,40671,8654,8528,1,2,4,6,8,2,4,6,8,4,6,1,2,6,8,4,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,10,10,36374,30966.0,23818,37277,30687.0,36394,37169.0,24223.0,24773.0,34543.0,23139.0,34421,34987.0,35472.0,111865,25005.0,35327.0,25150.0,97988.0,41877.0,127857.0,34466.0,5,7,7,15,14,13,2,1,0,0,15,27,6,10,52.0,48.0,2,1,2
4,2008/2009,1,2008-08-17 00:00:00,34633,10252,8456,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,1,3,5,7,9,5,1,3,3,3,3,7,7,7,7,10,10,1,3,3,3,3,7,7,7,7,7,11,30380,30357.0,24658,43280,23282.0,38609,24780.0,23782.0,23354.0,23264.0,26165.0,31432,46403.0,24208.0,23939,33963.0,47413.0,40198.0,42119.0,NaN,33633.0,107216.0,5,9,4,5,11,13,0,1,0,0,16,16,7,8,52.0,48.0,4,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2014/2015,9,2014-10-25 00:00:00,48955,8472,9825,1,2,4,6,8,5,2,4,6,8,5,1,2,4,6,8,4,6,3,5,7,5,1,3,3,3,3,6,8,8,8,8,11,1,3,3,3,3,6,6,8,8,8,11,37233,188540.0,24150,30362,180330.0,25150,24159.0,35443.0,109058.0,114218.0,32627.0,169718,425255.0,35606.0,38521,78513.0,23688.0,24011.0,50047.0,37436.0,196386.0,113836.0,4,5,3,9,9,8,3,3,0,0,14,21,3,3,39.0,61.0,0,2,0
2656,2014/2015,9,2014-10-25 00:00:00,38293,10003,8197,1,2,4,6,8,4,6,3,5,7,5,1,2,4,6,8,4,6,3,5,7,5,1

In [8]:
teams_data = epl_teams.select_dtypes(include = ['int64','float64'])
teams_data = teams_data.dropna()

In [9]:
teams_data

,team_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
4,9825,59,51.0,26,28,55,64,51,44,52
5,9825,59,51.0,30,28,44,46,51,44,52
10,10252,66,32.0,72,63,48,56,39,42,52
11,10252,63,37.0,54,60,48,38,35,44,54
16,8658,56,39.0,79,77,71,56,42,48,54
...,...,...,...,...,...,...,...,...,...,...
191,8654,77,37.0,57,68,73,29,30,37,41
196,8528,51,36.0,51,52,55,57,62,57,44
197,8528,51,37.0,56,52,59,40,62,48,44
202,8602,47,28.0,58,55,64,56,47,48,52


In [10]:
def gen_data(team_id):
    team = teams_data[teams_data['team_id']==team_id]
    team = team.loc[:,'buildUpPlaySpeed':].mean()
    team = team.to_dict()
    return team

In [11]:
def gen(df):
    home_team_ids = df.home_team_id
    away_team_ids = df.away_team_id
    l_home_teams = []
    l_away_teams = []
    for i in home_team_ids:
        l_home_teams.append(gen_data(i))
    for j in away_team_ids:
        l_away_teams.append(gen_data(j))
    l_home_teams = pd.DataFrame(l_home_teams)
    l_away_teams = pd.DataFrame(l_away_teams)
    l_home_teams = l_home_teams.rename(columns = {'buildUpPlaySpeed':'home_buildUpPlaySpeed','buildUpPlayDribbling':'home_buildUpPlayDribbling','buildUpPlayPassing':'home_buildUpPlayPassing','chanceCreationPassing':'home_chanceCreationPassing','chanceCreationCrossing':'home_chanceCreationCrossing','chanceCreationShooting':'home_chanceCreationShooting','defencePressure':'home_defencePressure','defenceAggression':'home_defenceAggression','defenceTeamWidth':'home_defenceTeamWidth'})
    l_away_teams = l_away_teams.rename(columns = {'buildUpPlaySpeed':'away_buildUpPlaySpeed','buildUpPlayDribbling':'away_buildUpPlayDribbling','buildUpPlayPassing':'away_buildUpPlayPassing','chanceCreationPassing':'away_chanceCreationPassing','chanceCreationCrossing':'away_chanceCreationCrossing','chanceCreationShooting':'away_chanceCreationShooting','defencePressure':'away_defencePressure','defenceAggression':'away_defenceAggression','defenceTeamWidth':'away_defenceTeamWidth'})
    return l_home_teams,l_away_teams

In [12]:
home_team_train,away_team_train = gen(epl_matches_train)
train_data = home_team_train.join(away_team_train)
train_data_matrix = train_data.to_numpy()
train_label = np.array(list(epl_matches_train.results))

In [13]:
home_team_test,away_team_test = gen(epl_matches_test)
test_data = home_team_test.join(away_team_test)
test_data_matrix = test_data.to_numpy()

In [14]:
model_l = [SVC(),LogisticRegression(multi_class='multinomial')]
model_acc = []
model_params = []
for i in range(2):
    clf = model_l[i]
    param_grid = {'C': 10. ** np.arange(-6, 4)}
    grid_search = GridSearchCV(clf, param_grid=param_grid, cv=20, verbose=3, return_train_score=True)
    grid_search.fit(train_data_matrix, train_label)
    model_acc.append(grid_search.best_score_)
    model_params.append(grid_search.best_params_)

Fitting 20 folds for each of 10 candidates, totalling 200 fits
[CV] C=1e-06 .........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ......... C=1e-06, score=(train=0.463, test=0.466), total=   0.6s
[CV] C=1e-06 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] ......... C=1e-06, score=(train=0.463, test=0.466), total=   0.4s
[CV] C=1e-06 .........................................................


...

C:\Users\subhash chandran\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] ........ C=1000.0, score=(train=0.524, test=0.602), total=   0.2s
[CV] C=1000.0 ........................................................
[CV] ........ C=1000.0, score=(train=0.531, test=0.496), total=   0.1s
[CV] C=1000.0 ........................................................


C:\Users\subhash chandran\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\subhash chandran\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

[CV] ........ C=1000.0, score=(train=0.527, test=0.526), total=   0.2s
[CV] C=1000.0 ........................................................
[CV] ........ C=1000.0, score=(train=0.529, test=0.474), total=   0.2s


C:\Users\subhash chandran\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
sgd = SGDClassifier(max_iter=1000)
param_grid2 = {
    "loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "none"],
}
grid_search2 = GridSearchCV(sgd, param_grid=param_grid2, cv=10, verbose=3, return_train_score=True)
grid_search2.fit(train_data_matrix,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 48 candidates, totalling 480 fits
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.498, test=0.470), total=   0.4s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.498, test=0.485), total=   0.1s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.480, test=0.481), total=   0.1s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.254, test=0.256), total=   0.1s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.254, test=0.256), total=   0.1s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.528, test=0.511), total=   0.1s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.254, test=0.252), total=   0.1s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hinge, penalty=l2, score=(train=0.518, test=0.549), total=   0.1s
[CV] alpha=0.0001, loss=hinge, penalty=l2 ............................
[CV]  alpha=0.0001, loss=hin

[CV]  alpha=0.0001, loss=log, penalty=none, score=(train=0.371, test=0.346), total=   0.2s
[CV] alpha=0.0001, loss=log, penalty=none ............................
[CV]  alpha=0.0001, loss=log, penalty=none, score=(train=0.518, test=0.500), total=   0.2s
[CV] alpha=0.0001, loss=log, penalty=none ............................
[CV]  alpha=0.0001, loss=log, penalty=none, score=(train=0.485, test=0.481), total=   0.2s
[CV] alpha=0.0001, loss=log, penalty=none ............................
[CV]  alpha=0.0001, loss=log, penalty=none, score=(train=0.398, test=0.436), total=   0.2s
[CV] alpha=0.0001, loss=log, penalty=none ............................
[CV]  alpha=0.0001, loss=log, penalty=none, score=(train=0.479, test=0.470), total=   0.3s
[CV] alpha=0.0001, loss=log, penalty=none ............................
[CV]  alpha=0.0001, loss=log, penalty=none, score=(train=0.464, test=0.466), total=   0.2s
[CV] alpha=0.0001, loss=log, penalty=none ............................
[CV]  alpha=0.0001, loss=log

...

[CV]  alpha=0.1, loss=modified_huber, penalty=l2, score=(train=0.486, test=0.481), total=   0.1s
[CV] alpha=0.1, loss=modified_huber, penalty=l2 ......................
[CV]  alpha=0.1, loss=modified_huber, penalty=l2, score=(train=0.465, test=0.462), total=   0.1s
[CV] alpha=0.1, loss=modified_huber, penalty=l1 ......................
[CV]  alpha=0.1, loss=modified_huber, penalty=l1, score=(train=0.510, test=0.496), total=   0.1s
[CV] alpha=0.1, loss=modified_huber, penalty=l1 ......................
[CV]  alpha=0.1, loss=modified_huber, penalty=l1, score=(train=0.474, test=0.481), total=   0.1s
[CV] alpha=0.1, loss=modified_huber, penalty=l1 ......................
[CV]  alpha=0.1, loss=modified_huber, penalty=l1, score=(train=0.361, test=0.368), total=   0.1s
[CV] alpha=0.1, loss=modified_huber, penalty=l1 ......................
[CV]  alpha=0.1, loss=modified_huber, penalty=l1, score=(train=0.495, test=0.492), total=   0.1s
[CV] alpha=0.1, loss=modified_huber, penalty=l1 ...............

C:\Users\subhash chandran\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


[CV]  alpha=0.1, loss=modified_huber, penalty=none, score=(train=0.442, test=0.444), total=   0.4s
[CV] alpha=0.1, loss=modified_huber, penalty=none ....................
[CV]  alpha=0.1, loss=modified_huber, penalty=none, score=(train=0.504, test=0.470), total=   0.2s
[CV] alpha=0.1, loss=modified_huber, penalty=none ....................
[CV]  alpha=0.1, loss=modified_huber, penalty=none, score=(train=0.261, test=0.271), total=   0.1s
[CV] alpha=0.1, loss=modified_huber, penalty=none ....................
[CV]  alpha=0.1, loss=modified_huber, penalty=none, score=(train=0.329, test=0.305), total=   0.2s
[CV] alpha=0.1, loss=modified_huber, penalty=none ....................
[CV]  alpha=0.1, loss=modified_huber, penalty=none, score=(train=0.502, test=0.459), total=   0.2s
[CV] alpha=0.1, loss=modified_huber, penalty=none ....................
[CV]  alpha=0.1, loss=modified_huber, penalty=none, score=(train=0.429, test=0.372), total=   0.2s
[CV] alpha=0.1, loss=modified_huber, penalty=none .

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  1.3min finished


GridSearchCV(cv=10, estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'modified_huber'],
                         'penalty': ['l2', 'l1', 'none']},
             return_train_score=True, verbose=3)

In [16]:
Model = ['SVC','LogisticRegression','SGDClassifier']
Model_accuracy = model_acc+[grid_search2.best_score_]
Model_params = model_params+[grid_search2.best_params_]
df = {'Model':Model,'Accuracy':Model_accuracy,'Params':Model_params}
df = pd.DataFrame(df)
df

,Model,Accuracy,Params
0,SVC,0.530451,{'C': 1.0}
1,LogisticRegression,0.523308,{'C': 0.001}
2,SGDClassifier,0.494737,"{'alpha': 0.1, 'loss': 'log', 'penalty': 'none'}"


In [17]:
best_model = LogisticRegression(C=0.001)
best_model.fit(train_data_matrix,train_label)
prediction = best_model.predict(test_data_matrix)
pred = {'match_id':list(epl_matches_test['match_id']),'result':prediction}
pred = pd.DataFrame(pred)
map = {0:'lose', 1:'draw', 2:'win'}
pred.replace(map,inplace = True)

C:\Users\subhash chandran\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
pred.to_csv('prediction_submission.csv')

In [846]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [847]:
def split(X,y,ratio):
    permutation = np.array([i for i in range(len(X))])
    np.random.shuffle(permutation)
    X = np.array([X[i] for i in permutation])
    y = np.array([y[i] for i in permutation])
    split_index = int(ratio*X.shape[0])
    train_data = X[:split_index]
    train_label = y[:split_index]
    valid_data = X[split_index:]
    valid_label = y[split_index:]
    return train_data,train_label,valid_data,valid_label

In [848]:
def batchify(X,y,batch_size):
    # Only take batch_size chunks (i.e. drop the remainder)
    N = int(len(X) / batch_size) * batch_size
    batches = []
    for i in range(0, N, batch_size):
        batches.append({
            'x': torch.tensor(X[i:i+batch_size], dtype=torch.float32),
            'y': torch.tensor(y[i:i+batch_size], dtype=torch.long
        )})
    return batches

In [849]:
class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(18,10)
        self.fc2 = nn.Linear(10,3)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [850]:
def train_model(split_ratio,batch_size,learning_rate,n_epochs):
    nn_train_data,nn_train_label,nn_valid_data,nn_valid_label = split(train_data_matrix,train_label,split_ratio)
    permutation = np.array([i for i in range(len(nn_train_data))])
    np.random.shuffle(permutation)
    nn_train_data = np.array([nn_train_data[i] for i in permutation])
    nn_train_label = np.array([nn_train_label[i] for i in permutation])


    train_batches = batchify(nn_train_data,nn_train_label,batch_size)
    valid_batches = batchify(nn_valid_data,nn_valid_label,batch_size)


    model = NN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(),lr=learning_rate)


#     total = 0
#     count = {0:0,1:1,2:2}
#     for batch in train_batches:
#         for y in batch['y']:
#             count[int(y)]+=1
#             total+=1
#     for i in count:
#         print(f"{i} : {count[i]/total*100}")



    for epoch in range(n_epochs):
        for batch in train_batches:
            data = batch['x']
            label = batch['y']
            scores = model(data)
            loss = criterion(scores,label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return valid_batches,model

In [851]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for batch in valid_batches:
            X = batch['x']
            y = batch['y']
            scores = model(X)
            predictions = torch.argmax(scores, dim =1)
            num_correct+= (predictions == y).sum()
            num_samples+= predictions.size(0)
        accuracy = float(num_correct)/float(num_samples)*100
        return accuracy

In [870]:
inp,mod = train_model(0.8,512,0.001,400)
accuracy = check_accuracy(inp,mod)    
print(accuracy)

48.95833333333333
